# Linear Regression with Multiple Features

On the previous class, we looked on how to create a linear regression pipeline using a single feature. Now, we will see how to apply a linear regression with multiple features.

First, let's import some modules:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

For the tests, we will use the same dataset from the last lecture, the Boston house pricing:

In [ ]:
data = pd.read_csv('../data/BostonHousing.csv')
data.head()

In [ ]:
np.random.seed(666)
msk = np.random.rand(len(data)) < 0.8 # Create a mask with random indexes

train = data[msk]
test = data[~msk]

print(len(data), len(train), len(test), len(train) + len(test))

Let's select a list of features and target for the predictions:

In [ ]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'yr_built', 'condition']
target = 'price'

The first thing to look at is, instead of an analytical solution, we will need to find an approximate solution.

The features can be represented by a set of arrays $x_{i}[j]$, where $j$ is the feature and $i$ is a feature element. The "linear" equation (now a multi-dimensional plane) will have the intercept and a coefficient for each feature, and is written as:

$$y_{i}^{,}(x_i) = w_0*x_{i}[0] + w_{1}*x_{i}[1] + w_{2}*x_{i}[2] + \dots + w_{d}*x_{i}[d] = \sum_{j=0}^{d}w_j*x_{i}[j]$$

where $x_i[0]=1$. We can use matrix notation and use the dot product of the parameters $\textbf{w}$ with the features $\textbf{x}_i$:

$$y_{i}^{,}(\textbf{x}_i) = \textbf{w}^{T}\textbf{x}_i$$

As for the linear regression with a single feature, the solution for multiple features come from the minimization of the ***Residual Sum of Squares*** (RSS):

$$RSS(\textbf{w}) = \sum_{i=1}^{N}(y_i - y_{i}^{,})^2 = \sum_{i=1}^{N}(y_i - \textbf{w}^{T}\textbf{x}_i)^2 = (\textbf{y} - \textbf{w}^{T}\textbf{x})^{T}(\textbf{y} - \textbf{w}^{T}\textbf{x})$$

The gradient $\Phi$ (the derivative of RSS relative to $\textbf{w}$) is:

$$\Phi = -2\textbf{x}^{T}(\textbf{y} - \textbf{w}^{T}\textbf{x})$$

In others words, we just need to take the dot product between the features and the residuals (the difference between the predictions and the measured data) and multiply by 2. The gradient is used to update the parameters $\textbf{w}$ iteratively in a way to minimize the errors (set the gradient to zero). In the [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) method, for the n-th iteration, we end up with the following equation:

$$\textbf{w}_{n+1} = \textbf{w}_{n} - \alpha\Phi = \textbf{w}_{n} + 2{\alpha}\textbf{x}^{T}(\textbf{y} - \textbf{w}^{T}_{n}\textbf{x})$$

where $\alpha$ is the learning rate, a scalar that helps the minimization to converge faster to the minimum point. The choice of the step length shows to be trick, as a large value wil lead to minimization to diverge as a too small value will diverge slowly.

Now we shall be able to compute the best set of parameters given a set of features and the output target. But to compute this, first we need to convert the data in the sframe format (the Turicreate dataframe) to a numpy numerical matrix. We will create a series of functions to combine in a final one that will calculate the parameters.

To facilitate the operations, let's convert the columns of the dataframe to numpy arrays:

In [ ]:
def get_numpy_data(df, features, output):
    # First, create a column with the feature x_0 = 1
    df['constant'] = 1

    # Include the w_0 in the features list in the first column
    features = ['constant'] + features 
    
    # From the data, pick only desired features and convert to numpy array
    feature_matrix = df[features].values
    
    # Doing the same for the output
    output_array = df[output].values
    
    # Return the features and output target as numerical matrices
    return(feature_matrix, output_array)

Let's test the function:

In [ ]:
(example_features, example_output) = get_numpy_data(data, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print(example_features[0,:])
print(example_output[0])

Now, to make our life easier, we can use the fact that the predictions are simply the **dot product** of the **weights** with the **features**, to create a function to compute the prediction given the matrix of the features and the array of parameters (weights). For that, we will the *Numpy* function [dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html).

In [ ]:
def predict_output(feature_matrix, weights):
    # The predictions are the dot product of the features and weights (parameters)
    predictions = ### YOUR CODE HERE ###
    
    # Return the predictions
    return(predictions)

Testing...

In [ ]:
# First computing manually
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)

# Now using our function
test_predictions = predict_output(example_features, my_weights)

print(predicted_value)
print(test_predictions[0])
print(test_predictions[1])

Now, let's create a function that, by giving the residuals and features, it returns the gradient $\Phi$ (the derivative of RSS relative to $\textbf{w}$):

$$\Phi = -2\textbf{x}^{T}(\textbf{y} - \textbf{w}^{T}\textbf{x})$$

In [ ]:
def feature_derivative(residuals, feature):
    # Using the equation of the derivative, do the dot product between the features and residuals
    derivative = ### YOUR CODE HERE ###
    
    # Return the derivative
    return(derivative)

Another testing...

In [ ]:
# Using only on feature
(example_features, example_output) = get_numpy_data(data, ['sqft_living'], 'price') 

# Making the parameters = 0 so the derivative should be
# equal the sum of the output. This is an easy way to
# evaluate our function
my_weights = np.array([0., 0.])

test_predictions = predict_output(example_features, my_weights) 

residuals = test_predictions - example_output

# Let's compute the derivative with respect to 'constant',
# the ":" indicates "all rows"
features = example_features[:,0] 

derivative = feature_derivative(residuals, features)

print(derivative)
print(-np.sum(example_output)*2) # should be the same as derivative
print(derivative - (-np.sum(example_output)*2)) # should be ZERO

Perfect. Now, let's head to the to the gradient descent method. 

First, we need to define a stoping criteria (tolerace), a value that stops the loop if the gradient is smaller than it. Also, it needs the initial parameters (weights) guess, and the step length (set as constant for all the iterations) as inputs.

In [ ]:
from math import sqrt

In [ ]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    iteration = 0
    
    while not converged:
        # Compitung predictions with current weights
        predictions = predict_output(feature_matrix, weights) 
        
        # Computing current residuals
        residuals = predictions-output
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Estimating the derivative for each parameter (weight)
            derivative = feature_derivative(residuals,feature_matrix[:, i])
            
            # add the squared value of the derivative to the gradient magnitude (for assessing convergence)
            gradient_sum_squares = gradient_sum_squares + (derivative*derivative)
            
            # subtract the step size times the derivative from the current weight
            weights[i] = weights[i] - step_size*derivative
        
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        iteration = iteration+1

        # The tolerance is our stoping criteria
        if gradient_magnitude < tolerance:
            converged = True
    
    # Return the parameters (weights) and number of iterations
    return(weights,iteration)

Testing:

In [ ]:
# Let's run the function over the train data
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

# Gradient descent
(weights,iteration) = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)

# Predictions over the test data
(test_feature_matrix, test_output_new) = get_numpy_data(test, model_features, my_output)
prediction = predict_output(test_feature_matrix, weights)

# Checking outputs
print(weights)
print(iteration)
print(prediction[0])
print(output[0])
print(prediction[0] - test_output_new[0])